# Optimal Execution: Almgren-Chriss Framework

## Problem Statement
Execute a large order while minimizing a combination of:
1. **Market impact costs** - Price moves against us as we trade
2. **Timing risk** - Uncertainty in future prices

## The Almgren-Chriss Model

**Execution trajectory** $n(t)$: shares remaining at time $t$

**Cost components**:
- **Permanent impact**: $\gamma \sum_j x_j$ (proportional to total volume)
- **Temporary impact**: $\epsilon \sum_j x_j^2$ (quadratic in trade size)
- **Volatility risk**: Exposure to price uncertainty

**Objective**: Minimize expected cost + $\lambda \times$ variance

$$\min_{x} \mathbb{E}[\text{cost}] + \lambda \cdot \text{Var}[\text{cost}]$$

where $\lambda$ is the **risk aversion parameter**.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve
import seaborn as sns

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Model Parameters

In [ ]:
# Trading parameters
X = 1_000_000  # Total shares to execute
T = 1.0        # Time horizon (days)
N = 10         # Number of time intervals
dt = T / N     # Time step

# Market impact parameters
gamma = 0.1e-6   # Permanent impact coefficient ($/share^2)
epsilon = 0.5e-6  # Temporary impact coefficient ($/share^2)
sigma = 0.30     # Daily volatility (30%)

# Risk aversion
lambda_risk = 2e-6  # Risk aversion parameter

print(f"Execution Parameters:")
print(f"  Total shares: {X:,}")
print(f"  Time horizon: {T} days")
print(f"  Number of slices: {N}")
print(f"\nMarket Impact:")
print(f"  Permanent impact γ: {gamma:.2e}")
print(f"  Temporary impact ε: {epsilon:.2e}")
print(f"  Volatility σ: {sigma*100:.1f}%")
print(f"\nRisk Aversion λ: {lambda_risk:.2e}")

## 2. Optimal Execution Strategy

In [ ]:
def compute_optimal_trajectory(X, T, N, gamma, epsilon, sigma, lambda_risk):
    """
    Compute optimal execution trajectory using Almgren-Chriss.
    
    Returns:
    --------
    trajectory : array
        Holdings at each time point (n_0, n_1, ..., n_N)
    trades : array
        Trade sizes at each interval (x_1, x_2, ..., x_N)
    """
    dt = T / N
    
    # Compute kappa (trade rate decay parameter)
    kappa = np.sqrt(lambda_risk * sigma**2 / (epsilon * dt))
    
    # Compute trajectory
    trajectory = np.zeros(N + 1)
    trajectory[0] = X
    
    # Analytical solution from Almgren-Chriss
    sinh_kappa_T = np.sinh(kappa * T)
    
    for j in range(N):
        t_j = j * dt
        trajectory[j] = X * np.sinh(kappa * (T - t_j)) / sinh_kappa_T
    
    trajectory[N] = 0  # All shares executed at final time
    
    # Compute trade sizes
    trades = -np.diff(trajectory)
    
    return trajectory, trades

# Compute optimal strategy
trajectory_opt, trades_opt = compute_optimal_trajectory(
    X, T, N, gamma, epsilon, sigma, lambda_risk
)

print("Optimal Execution Schedule:")
print(f"\n{'Time':<10} {'Holdings':<15} {'Trade Size':<15} {'% of Total':<15}")
print("-" * 60)
times = np.linspace(0, T, N + 1)
for i, (t, n) in enumerate(zip(times, trajectory_opt)):
    if i < N:
        trade = trades_opt[i]
        pct = (trade / X) * 100
        print(f"{t:<10.3f} {n:<15,.0f} {trade:<15,.0f} {pct:<15.2f}%")
    else:
        print(f"{t:<10.3f} {n:<15,.0f} {'---':<15} {'---':<15}")

## 3. Compare Strategies

In [ ]:
# Benchmark strategies

# 1. TWAP (Time-Weighted Average Price) - uniform trading
trajectory_twap = np.linspace(X, 0, N + 1)
trades_twap = np.diff(trajectory_twap) * -1

# 2. Front-loaded (aggressive early execution)
trajectory_front = X * (1 - np.linspace(0, 1, N + 1)**2)
trades_front = np.diff(trajectory_front) * -1

# 3. Back-loaded (delay execution)
trajectory_back = X * (np.linspace(1, 0, N + 1)**2)
trades_back = np.diff(trajectory_back) * -1

# Plot trajectories
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Holdings trajectory
times = np.linspace(0, T, N + 1)
axes[0].plot(times, trajectory_opt, 'o-', label='Optimal (AC)', linewidth=2, markersize=6)
axes[0].plot(times, trajectory_twap, 's--', label='TWAP', linewidth=2, markersize=6)
axes[0].plot(times, trajectory_front, '^-.', label='Front-loaded', linewidth=2, markersize=6)
axes[0].plot(times, trajectory_back, 'v:', label='Back-loaded', linewidth=2, markersize=6)
axes[0].set_xlabel('Time (days)', fontsize=12)
axes[0].set_ylabel('Shares Remaining', fontsize=12)
axes[0].set_title('Execution Trajectories', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)
axes[0].set_xlim([0, T])

# Right: Trade sizes
trade_times = np.linspace(dt/2, T - dt/2, N)
width = dt * 0.2
axes[1].bar(trade_times - 1.5*width, trades_opt, width, label='Optimal', alpha=0.8)
axes[1].bar(trade_times - 0.5*width, trades_twap, width, label='TWAP', alpha=0.8)
axes[1].bar(trade_times + 0.5*width, trades_front, width, label='Front', alpha=0.8)
axes[1].bar(trade_times + 1.5*width, trades_back, width, label='Back', alpha=0.8)
axes[1].set_xlabel('Time (days)', fontsize=12)
axes[1].set_ylabel('Trade Size (shares)', fontsize=12)
axes[1].set_title('Trade Sizes by Strategy', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('execution_strategies.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Cost Analysis

In [ ]:
def compute_execution_cost(trades, gamma, epsilon, sigma, dt):
    """
    Compute expected cost and variance for a given execution schedule.
    
    Returns:
    --------
    expected_cost : float
        Expected execution cost
    variance : float
        Variance of execution cost
    """
    # Permanent impact cost
    permanent_cost = gamma * np.sum(trades) * np.sum(trades)
    
    # Temporary impact cost
    temporary_cost = epsilon * np.sum(trades**2)
    
    # Expected cost
    expected_cost = permanent_cost + temporary_cost
    
    # Variance from timing risk
    # Holdings at each time contribute to variance
    holdings = np.cumsum(trades[::-1])[::-1]  # Remaining shares at each step
    variance = (sigma**2 * dt) * np.sum(holdings**2)
    
    return expected_cost, variance

# Compute costs for all strategies
strategies = {
    'Optimal (AC)': trades_opt,
    'TWAP': trades_twap,
    'Front-loaded': trades_front,
    'Back-loaded': trades_back
}

results = {}
for name, trades in strategies.items():
    exp_cost, var_cost = compute_execution_cost(trades, gamma, epsilon, sigma, dt)
    total_cost = exp_cost + lambda_risk * var_cost
    results[name] = {
        'expected': exp_cost,
        'variance': var_cost,
        'total': total_cost
    }

# Display results
print("\nCost Comparison:")
print(f"\n{'Strategy':<20} {'Expected Cost':<20} {'Variance':<20} {'Total Cost':<20}")
print("-" * 80)
for name, costs in results.items():
    print(f"{name:<20} ${costs['expected']:<19,.2f} ${costs['variance']:<19,.2f} ${costs['total']:<19,.2f}")

# Compute improvement over TWAP
twap_total = results['TWAP']['total']
opt_total = results['Optimal (AC)']['total']
improvement = ((twap_total - opt_total) / twap_total) * 100

print(f"\nOptimal strategy saves {improvement:.2f}% vs TWAP")
print(f"Absolute savings: ${twap_total - opt_total:,.2f}")

## 5. Risk-Aversion Sensitivity

In [ ]:
# Test different risk aversion levels
lambda_values = np.logspace(-7, -5, 20)
total_costs = []
expected_costs = []
variances = []

for lam in lambda_values:
    traj, trades = compute_optimal_trajectory(X, T, N, gamma, epsilon, sigma, lam)
    exp_cost, var_cost = compute_execution_cost(trades, gamma, epsilon, sigma, dt)
    total_costs.append(exp_cost + lam * var_cost)
    expected_costs.append(exp_cost)
    variances.append(var_cost)

# Plot efficient frontier
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Risk-return tradeoff
axes[0].plot(np.sqrt(variances), expected_costs, 'o-', linewidth=2, markersize=6)
axes[0].set_xlabel('Std Dev of Cost ($)', fontsize=12)
axes[0].set_ylabel('Expected Cost ($)', fontsize=12)
axes[0].set_title('Efficient Frontier: Cost vs Risk', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Mark current strategy
idx = np.argmin(np.abs(lambda_values - lambda_risk))
axes[0].plot(np.sqrt(variances[idx]), expected_costs[idx], 'r*', 
             markersize=15, label=f'Current λ={lambda_risk:.2e}')
axes[0].legend(fontsize=10)

# Right: Total cost vs lambda
axes[1].semilogx(lambda_values, total_costs, 'o-', linewidth=2, markersize=6)
axes[1].axvline(lambda_risk, color='red', linestyle='--', linewidth=2, 
                label=f'Current λ={lambda_risk:.2e}')
axes[1].set_xlabel('Risk Aversion λ', fontsize=12)
axes[1].set_ylabel('Total Cost ($)', fontsize=12)
axes[1].set_title('Total Cost vs Risk Aversion', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('risk_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Monte Carlo Simulation

In [ ]:
def simulate_execution(trades, S0, sigma, gamma, epsilon, dt, n_simulations=1000):
    """
    Simulate realized execution costs under random price paths.
    """
    np.random.seed(42)
    N = len(trades)
    costs = np.zeros(n_simulations)
    
    for sim in range(n_simulations):
        # Simulate price path
        returns = np.random.normal(0, sigma * np.sqrt(dt), N)
        price_path = S0 * np.exp(np.cumsum(returns))
        
        # Compute execution cost
        execution_prices = price_path + gamma * np.cumsum(trades) + epsilon * trades
        cost = np.sum(trades * execution_prices)
        costs[sim] = cost
    
    return costs

# Run simulations
S0 = 100  # Initial stock price
n_sims = 10000

costs_opt = simulate_execution(trades_opt, S0, sigma, gamma, epsilon, dt, n_sims)
costs_twap = simulate_execution(trades_twap, S0, sigma, gamma, epsilon, dt, n_sims)

# Plot distributions
plt.figure(figsize=(12, 6))
plt.hist(costs_opt, bins=50, alpha=0.7, label='Optimal (AC)', density=True, edgecolor='black')
plt.hist(costs_twap, bins=50, alpha=0.7, label='TWAP', density=True, edgecolor='black')
plt.axvline(np.mean(costs_opt), color='blue', linestyle='--', linewidth=2, 
            label=f'Optimal Mean: ${np.mean(costs_opt):,.0f}')
plt.axvline(np.mean(costs_twap), color='orange', linestyle='--', linewidth=2,
            label=f'TWAP Mean: ${np.mean(costs_twap):,.0f}')
plt.xlabel('Total Execution Cost ($)', fontsize=12)
plt.ylabel('Probability Density', fontsize=12)
plt.title(f'Distribution of Execution Costs ({n_sims:,} simulations)', 
          fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.savefig('cost_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nMonte Carlo Results ({n_sims:,} simulations):")
print(f"\n{'Strategy':<15} {'Mean Cost':<20} {'Std Dev':<20} {'5% VaR':<20}")
print("-" * 75)
print(f"{'Optimal':<15} ${np.mean(costs_opt):<19,.2f} ${np.std(costs_opt):<19,.2f} "
      f"${np.percentile(costs_opt, 95):<19,.2f}")
print(f"{'TWAP':<15} ${np.mean(costs_twap):<19,.2f} ${np.std(costs_twap):<19,.2f} "
      f"${np.percentile(costs_twap, 95):<19,.2f}")

## 7. Key Insights

**Almgren-Chriss Model Behavior**:
1. **Exponential decay**: Optimal strategy follows $n(t) \propto \sinh(\kappa(T-t))$
2. **Front-loading**: Higher risk aversion → more aggressive early execution
3. **Impact vs Risk**: Balance permanent/temporary impact against timing risk

**Practical Extensions**:
- **Time-varying volatility**: Adjust $\sigma(t)$ intraday (higher at open/close)
- **Market orders vs limits**: Model fill probability and adverse selection
- **Dark pools**: Reduce temporary impact, increase uncertainty
- **Multi-asset execution**: Correlation between positions matters

**Implementation Challenges**:
1. **Parameter estimation**: $\gamma$, $\epsilon$ vary by stock, time of day
2. **Regime changes**: Model may break during news events
3. **Slippage**: Real execution deviates from model predictions
4. **Latency**: Optimal trajectory must update in real-time

**Advanced Topics**:
- Reinforcement learning for adaptive execution
- Obizhaeva-Wang model (accounts for order book resilience)
- Kyle's Lambda for informed trading